# Analyze eBPF Programs

This notebook compiles, analyzes, and generates the report for eBPF programs. 

The raw report is saved in the `output/report` directory.

In [1]:
%reload_ext autoreload
%autoreload 2

The following cell compiles the BCC tools. 

In [2]:
from utils import BCC_TOOLS_PATH

!make -C {BCC_TOOLS_PATH} -j $(nproc)

make: Entering directory '/Users/szhong/Code/DepSurf/software/bcc/libbpf-tools'
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Code/DepSurf/software/bcc/libbpf-tools'


The following cell compiles the eBPF program for Tracee.

In [3]:
from utils import TRACEE_PATH

!make -C {TRACEE_PATH} bpf

make: Nothing to be done for 'bpf'.


The following cell analyzes and generates the report for eBPF programs.

In [7]:
from depsurf import BPFProgram, VersionGroup, DepReport, DepKind
from utils import TRACEE_PATH, BCC_OBJ_PATH, OUTPUT_PATH, save_pkl

GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH]
PROG_PATHS = sorted(BCC_OBJ_PATH.glob("*.bpf.o")) + [
    TRACEE_PATH / "dist" / "tracee.bpf.o"
]


REPORT_KINDS = [
    DepKind.FUNC,
    DepKind.TRACEPOINT,
    DepKind.LSM,
    DepKind.FIELD,
    DepKind.STRUCT,
    DepKind.SYSCALL,
]

results = {}
for path in PROG_PATHS:
    bpf = BPFProgram(path)
    result_path = OUTPUT_PATH / "programs" / bpf.name
    result_path.mkdir(parents=True, exist_ok=True)
    prog_report = {}
    for dep in bpf.deps:
        if dep.kind not in REPORT_KINDS:
            continue
        with open(result_path / f"{dep.kind} {dep.name}.md", "w") as f:
            report = DepReport.from_groups(dep, GROUPS)
            report.print(file=f)
            prog_report[dep] = report
    print(f"Report saved to {result_path}", flush=True)
    results[bpf.name] = prog_report


save_pkl(results, "bcc")

Report saved to /Users/szhong/Code/DepSurf/data/output/programs/bashreadline
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/bindsnoop
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/biolatency
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/biopattern
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/biosnoop
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/biostacks
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/biotop
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/bitesize
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/cachestat
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/capable
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/cpudist
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/cpufreq
Report saved to /Users/szhong/Code/DepSurf/data/output/programs/drsnoop
Report saved to /Users/szhong/Code/DepSurf/dat

[ utils_pickle.py:13 ] INFO: Saved bcc to /Users/szhong/Code/DepSurf/data/output/bcc.pkl
